# Lab 2: Numpy
In this Numpy exercise, the general requirement is not to use loops; I will specify where it is allowed

(Last update: 12/11/2023)

Name: Đoàn Ngọc Mai 
Sdudent ID: 21127104

---

## 0. Instructions for doing and submitting assignment

**How to do your assignment**

You will do your assignment directly on this notebook file. First, you fill your name and student code at the beginning of the file. In this file, you will write your code when you see the following lines of code:
```python
# YOUR CODE HERE
raise NotImplementedError()
```

For optional coding parts, there will be:
```python
# YOUR CODE HERE (OPTION)
```

For markdown cell, there will be:
```markdown
YOUR ANSWER HERE
```

Of course, you have to remove the `raise NotImplementedError()` statement when you finish.

For coding parts, there are often cells below to help you check your answers. You will pass the test if there are no errors when you run the test cells. In some cases, the tests are insufficient. That means if you do not pass the test, your answer is definitely wrong somewhere, but if you pass the test, your answer may still be incorrect.

While doing the assignment, you should print out the output and create more cells for testing. But you have to remove all of them (comment your print-out codes, delete the cell created by you) when you submit your code. <font color=red>Do not remove or edit my cells</font> (except for the aforementioned cells).

Keep your code clean and clear by using meaningful variable names and comments, not write too-long coding lines.
Press `Ctrl + S` right after editing.

Keep it real: The reason why you are here is to <font color=green>study, really study</font>. I highly recommend that you discuss your idea with your friends and <font color=green>write your own code based on your own knowledge</font>. <font color=red>Copy means zero.</font>

**How to submit your assignment**

When grading your assignment, I will choose `Kernel` - `Restart & Run All` in order to restart the kernel and run all cells in your notebook. Therefore, you should do that before submitting to ensure that the outputs are all as expected.

After that, rename the notebook as `<Student ID>.ipynb`. For example, if your student code is 1234567, then your notebook is `1234567.ipynb`.

Finally, submit your notebook file on Moodle. <font color=red>Please strictly follow the submission rules.</font>

---

---

## 1. Programming environment

- You will re-use the Linux environment setup in Lab 0 - WarmUp. Don't forget to start your coding environment (`conda activate min_ds-env`) before doing your assignment.
- Use Jupyter notebook or Jupyter lab, <font color=red>not Google Colab</font> (I can not grade you well on Google Colab) to edit your `*.ipynb` file.

In [106]:
import sys
sys.executable

'c:\\Users\\ADMIN\\anaconda3\\envs\\min_ds-env\\python.exe'

- If there are no problems, the file to run python will be the file of the "min_ds-env" code environment.

- In this article, you are not using the Pandas library.

---

## 2. Import necessary libraries

In [107]:
import numpy as np
# YOUR CODE HERE (OPTION)

---

## 3. Data collection

Numpy is not a great library for handling operations like data reading and writing, but it's an excellent library for computational tasks. Therefore, in this article, we'll simply use the pre-collected dataset that I've attached in the folder of this lab. This dataset actually contains multiple files and is relatively large, but it has been curated to include relevant information for this lab. You can learn more about this dataset [here](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020).

Here is a specific description of the dataset:
>The full dataset have the list of all animes register by the user with the respective score, watching status and numbers of episodes watched. This dataset contains 109 Million row, 17.562 different animes and 325.772 different users. The file have the following columns:\
1 - user_id: non identifiable randomly generated user id.\
2 - anime_id: MyAnemlist ID of the anime. (e.g. 1).\
3 - score: score between 1 to 10 given by the user. 0 if the user didn't assign a score. (e.g. 10)\
4 - watching_status: state ID from this anime in the anime list of this user. (e.g. 2)\
5 - watched_episodes: numbers of episodes watched by the user. (e.g. 24)

---

## 4. Data exploring & Data preprocessing

### How many rows and columns does the data have?

Of course, the first thing you need to do is read the data file into the Numpy array and name it `user_ratings` (use function `np.genfromtxt`). You may encounter some minor problems with this task, it seems that all the data in the Numpy array is not what we want. This happens because the function `np.genfromtxt` has a default data type of `float`, you need to find a way to convert it to `uint64`. You should put the dataset file in the same directory as this notebook file to simplify when passing the file name to the function. Finally, you need to calculate the number of rows and columns for this dataset, these two values are stored in two variables `nrows` and `ncols` respectively.

In [108]:
# YOUR CODE HERE
# raise NotImplementedError()
user_ratings = np.genfromtxt('anime.txt', dtype=np.uint64, delimiter='\t')
nrows, ncols = user_ratings.shape

In [109]:
# TEST
assert user_ratings.dtype == np.uint64
assert user_ratings.ndim == 2
assert nrows == 66338
assert ncols == 5
user_ratings[:5] # Look at the first 5 lines

array([[ 866,  138,    0,    6,    0],
       [ 858,  696,    0,    6,    0],
       [ 617,  147,    0,    6,    0],
       [ 481, 1951,    8,    2,    3],
       [ 890,  690,    6,    2,   26]], dtype=uint64)

### Rows

#### The meaning of each row

Each row in the data set shows some information about a user's score for a anime.

#### Does the data have duplicate rows?

You will test this case and save the results to the `have_duplicated_rows` variable. This variable will have the value True if the data has duplicate lines and will have the value False otherwise.

In [110]:
# YOUR CODE HERE
# raise NotImplementedError()
unique_rows, counts = np.unique(user_ratings, axis=0, return_counts=True)
have_duplicated_rows = len(unique_rows) < len(user_ratings)

# print(have_duplicated_rows)

In [111]:
# TEST
assert have_duplicated_rows == False

Great, so there are no duplicate rows. Next we will explore the columns.

### Columns

#### The meaning of each column
- The first column indicates the user's ID (user_ID)
- The second column indicates the ID of the anime movie (anime_ID)
- The third column shows the score at which the user rated the movie (ratings)
- The fourth column indicates watching status (watching_status)
- The fifth column indicates the episode watched (watched_episodes)

#### What data type does each column currently have?

In [112]:
user_ratings.dtype

dtype('uint64')

At first glance, it seems that all columns are numeric. But in my opinion, the two columns `user_id` and `anime_id` should be classified into categorical groups. The reason for this is because both `user_id` and `anime_id` are simply identifiers and do not necessarily have an arithmetic relationship between the columns. Of course, this is just an objective perspective and not true for all cases, but to make it easier to work, in this lab we will agree with the above thought.

#### For each column with numeric datatype, how are the values distributed?

First, we need to see how many missing values the numeric columns have. This mission is quite 'difficult' ^^ so I will do it for you.

In [113]:
np.sum(np.isnan(user_ratings[:, 2:]), axis=0)

array([0, 0, 0])

Great, so all numeric columns don't have any missing values.

Now, your job is to calculate the min, Q1(25%), median, Q3(75%) and max of these numeric columns. You will need to use the `np.percentile` function to do this. Then, the all values of each column are saved respectively into 3 Numpy arrays namely `rating_percentile`, `status_percentile`, `episodes_percentile`. Also, make sure that these Numpy arrays all have the `uint64` data type so you don't get into trouble with the TEST cell.

In [114]:
# YOUR CODE HERE
# raise NotImplementedError()
numeric_columns = user_ratings[:, 2:]

rating_percentile = np.percentile(numeric_columns[:, 0], [0, 25, 50, 75, 100], interpolation='nearest').astype(np.uint64)
status_percentile = np.percentile(numeric_columns[:, 1], [0, 25, 50, 75, 100], interpolation='nearest').astype(np.uint64)
episodes_percentile = np.percentile(numeric_columns[:, 2], [0, 25, 50, 75, 100], interpolation='nearest').astype(np.uint64)

In [115]:
# TEST
assert np.array_equal(rating_percentile, np.array([0, 0, 5, 8, 10]))
assert np.array_equal(status_percentile, np.array([1, 2, 2, 6, 6]))
assert np.array_equal(episodes_percentile, np.array([0, 0, 1, 24, 36600]))

#### For each column with categorical datatype, how are the values distributed?

Just like with numeric columns, we need to see if two categorical columns have missing values? (This is difficult so let me do it for you :v )

In [116]:
np.sum(np.isnan(user_ratings[:, :2]), axis=0)

array([0, 0])

Your task is to, for each column, calculate a list of 5 numbers: the number of distinct values, the value that appears least with its corresponding count (total of 2 numbers), and the value that appears most with its corresponding count (total of 2 numbers). You should store the 2 lists calculated for 2 columns in two variables, namely `user_profile` and `anime_profile`. If multiple users rate the least number of movies, we will agree to choose the user with the smallest id. And vice versa, if many users rate the most movies, we will choose the user with the largest id.

In [117]:
# YOUR CODE HERE
# raise NotImplementedError()
user_ids, user_counts = np.unique(user_ratings[:, 0], return_counts=True)
min_user_count_idx = np.argmin(user_counts)
max_user_count_idx = np.argmax(user_counts)

user_profile = [
    len(user_ids),  
    user_ids[min_user_count_idx], 
    user_counts[min_user_count_idx],  
    user_ids[max_user_count_idx],  
    user_counts[max_user_count_idx]  
]

anime_ids, anime_counts = np.unique(user_ratings[:, 1], return_counts=True)
min_anime_count_idx = np.argmin(anime_counts)
max_anime_count_idx = np.argmax(anime_counts)

anime_profile = [
    len(anime_ids),  
    anime_ids[min_anime_count_idx],  
    anime_counts[min_anime_count_idx],  
    anime_ids[max_anime_count_idx],  
    anime_counts[max_anime_count_idx]  
]

In [118]:
assert user_profile == [898, # Có chừng này user
                        31,  # Đây là user rate ít movie nhất
                        1,  # và đó là chừng này movie
                        890, # Đây là user rate nhiều movie nhất
                        1138] # và đó là chừng này movie
assert anime_profile == [1801,#Có chừng này movie
                         115, #Đây là movie được ít user rate nhất
                         1,   #và đó là chừng này user
                         1535,  #Đây là movie được nhiều user rate nhất
                         701] #và đó là chừng này user

Incidentally, we need to check the maximum and minimum values of the two columns `user_id` and `anime_id`:

In [119]:
print('User id  - min & max:', 
      user_ratings[:, 0].min(), '&', user_ratings[:, 0].max()) 
print('Anime id - min & max:', 
      user_ratings[:, 1].min(), '&', user_ratings[:, 1].max()) 

User id  - min & max: 0 & 1000
Anime id - min & max: 1 & 2000


"It can be observed that for `user_id`, there is nothing unusual, but for `anime_id`, the first anime is numbered from 1 instead of 0 like `user_id`. Although this is not a significant issue, for convenience in the subsequent implementation, we need to normalize the `anime_id` column."

In [120]:
user_ratings[:, 1] = user_ratings[:, 1] - 1

---

## 5. Question

The previous section was just to warm you up before diving into the main content of this lab. Now, we have a bit better understanding of the dataset. We will attempt to pose meaningful questions and find answers using the data.

One interesting question to ask is: *For each different user, is it possible to recommend animes that the user has never watched before?*

Finding an answer to this question can be beneficial for both users and movie streaming service providers:
- Users: Users may want to watch a movie, but with so many options available, they may not know which one to choose. It would be convenient for users if the system could suggest a list of movies that they are likely to enjoy.
- Movie Streaming Service Providers: If the system makes good recommendations, it's more likely that users will watch and enjoy the movies. This, in turn, means users will continue to pay for the service.

---

### Preprocessing

First, we need to decide which information to use in building the anime recommendation system. Obviously, the columns `user_id`, `anime_id`, and `rating` are essential to perform this task. As for the two columns `watching_status` and `watched_episodes`, these columns can still have value in practice when building a recommendation model. However, for simplicity, we will temporarily set aside these two columns here.

Based on 3 columns, you need to create a 2D NumPy matrix named `ratings_mat`. In this matrix, the number of rows represents the number of users, while the number of columns represents the number of anime. So, `ratings_mat[i, j]` will represent the rating that `user_i` has given to `anime_j`. "For anime series that the user has not rated, the value will be 'NaN'."

In [121]:
# # YOUR CODE HERE
# # raise NotImplementedError()
num_users = np.max(user_ratings[:, 0]).astype(int)
num_anime = np.max(user_ratings[:, 1]).astype(int)

# Create a matrix of NaNs for the ratings, adding 1 to account for zero-based indexing
ratings_mat = np.full((num_users + 1, num_anime + 1), np.nan)

user_ids = user_ratings[:, 0]
anime_ids = user_ratings[:, 1]
scores = user_ratings[:, 2]

scores = np.where(scores == 0, np.nan, scores)

ratings_mat[user_ids, anime_ids] = scores

In [122]:
assert ratings_mat.shape == (1001, 2000)
missing_ratios = np.mean(np.isnan(ratings_mat))
assert missing_ratios.round(3) == 0.982

### Analyze data to answer the question

It would be much simpler if we used algorithms supported by other libraries. However, the main goal of this lab is to help you practice using the Numpy library. Therefore, you will have the opportunity to build a simple anime recommendation system from scratch using the provided data, utilizing only Numpy library. Please remember that Numpy doesn't favor loops, so you are only allowed to use loops where I explicitly permit.

In my opinion, there are two fundamental tasks in a anime recommendation system:

- First, you need to predict the ratings for animes that a user hasn't reviewed or watched yet.
- Second, you need to provide recommendations to users based on the top-rated animes that have been predicted.

It seems that the second task will become much simpler if we can accomplish the first task. One of the simplest ways to tackle task 1 is by computing the similarity between users and using this similarity to make predictions. However, there are some considerations to keep in mind. It's not feasible to compute similarity between all users at once, as it might lead to memory issues (even if you have enough memory, my computer is quite limited in that regard :<). One way to address this issue is to process a group of users at a time, referred to as `a batch`. To keep it simple, let's stick with a `batch_size = 32`, which I believe is a reasonable value. You should try to make your code work with a single batch first and then extend it to process all batches.

"First, you will try with a batch corresponding to users with indices from `start` to `end`."

In [123]:
batch_size = 32
start = 0
end = batch_size

Step 1: Calculate the `similarities` array to show the similarity between each user in the current batch with all users in the entire dataset. This array will have a size of `batch_size` x `n_users` (`n_users` is the total number of users in the dataset), where `similarities[i, j]` indicates the similarity between `user_i` and `user_j`. In the case where two users have no common rated movies (when running, you may see a warning 'RuntimeWarning: Mean of empty slice'), you set the similarity to 0.

In [124]:
np.set_printoptions(suppress = True)

In [125]:
# YOUR CODE HERE
# raise NotImplementedError()
r0 = ratings_mat[start:end, :]
a = np.abs(ratings_mat - r0[:, np.newaxis, :])
a = np.nanmean(a, axis=2)  # Ignore nan

similarities = 1 / (a + 0.001)  # Add 0.001 to avoid dividing by 0

similarities = np.nan_to_num(similarities, nan=0)
# similarities[:3, :3].round(1)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13924\3763824203.py:5: RuntimeWarning: Mean of empty slice
  a = np.nanmean(a, axis=2)  # Ignore nan


In [126]:
# TEST
assert similarities.shape == (32, 1001)
assert np.array_equal(similarities[:3, :3].round(1), 
                      np.array([[1.0e+03, 6.0e-01, 2.0e+00],
                                [6.0e-01, 1.0e+03, 1.5e+00],
                                [2.0e+00, 1.5e+00, 1.0e+03]]))

Step 2: calculate the `weights` matrix. The array `weights` will have the size `batch_size` x `n_users` x `n_movies` (where `n_movies` is the total number of movies). About how to calculate `weights`, you can refer to file `example.ipynb`.

When running, you will see the warning "RuntimeWarning: invalid value encountered in true_divide"; This is because the users who rate a movie under consideration all have a similarity of 0 with a user under review, resulting in normalization to 0/0 and the result is difficult. This case means there is not enough information to predict the score and in this article, you should leave it as it is.

In [127]:
# YOUR CODE HERE
# raise NotImplementedError()
weights = ~np.isnan(ratings_mat)*similarities.reshape(batch_size, -1, 1)
weights = weights/weights.sum(axis=1, keepdims=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13924\1399081430.py:4: RuntimeWarning: invalid value encountered in divide
  weights = weights/weights.sum(axis=1, keepdims=True)


In [128]:
# TEST
assert weights.shape == (32, 1001, 2000)
assert np.sum(np.isnan(weights)) == 16462446

Step 3: For each user in the batch under consideration, calculate the score (for all movies) by multiplying the score of all users with the corresponding weight in the `weight` array; then write each user's scores down to one line in the `filled_ratings` array.

In [129]:
filled_ratings = np.empty_like(ratings_mat)

In [130]:
# YOUR CODE HERE
# raise NotImplementedError()
filled_ratings[start:end] = np.nansum(ratings_mat * weights, axis=1)


In [131]:
# TEST
filled_batch = filled_ratings[start:end]
filled_nanvals = filled_batch[np.isnan(ratings_mat[start:end])]
assert np.array_equal(filled_nanvals[:3].round(1),
                      np.array([8.6, 0. , 0. ]))
assert np.array_equal(filled_nanvals[-3:].round(1),
                      np.array([0., 0., 0.]))

Great ! So your code has run on a batch, now it's time for you to use the `for` loop to cycle through all the batches in the data set.

In [134]:
# YOUR CODE HERE
# raise NotImplementedError()

num_batches = len(ratings_mat) // batch_size + int(len(ratings_mat) % batch_size > 0)
batches = np.array_split(ratings_mat, num_batches)
filled_ratings = np.empty((0, 2000), dtype=float)

for batch in batches:
    absolute_diff = np.abs(ratings_mat[:, None] - batch)
    similarities = 1 / (np.nanmean(absolute_diff, axis=-1) + 0.001)
    
    similarities = np.transpose(similarities)
    similarities[np.isnan(similarities)] = 0
    
    weight = ~np.isnan(ratings_mat) * similarities.reshape(len(batch), -1, 1)
    weight = weight / weight.sum(axis=1, keepdims=True)
    
    weighted_sum = np.nansum(ratings_mat * weight, axis=1)
    
    filled_ratings = np.append(filled_ratings, weighted_sum, axis=0)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13924\3132856473.py:10: RuntimeWarning: Mean of empty slice
  similarities = 1 / (np.nanmean(absolute_diff, axis=-1) + 0.001)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13924\3132856473.py:16: RuntimeWarning: invalid value encountered in divide
  weight = weight / weight.sum(axis=1, keepdims=True)


In [136]:
# TEST
filled_nanvals = filled_ratings[np.isnan(ratings_mat)]
assert np.array_equal(filled_nanvals[:3].round(1),
                      np.array([8.6, 0. , 0. ]))
assert np.array_equal(filled_nanvals[-3:].round(1),
                      np.array([5.7, 5. , 6.6]))